In [1]:
from osgeo import gdal
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from dbfread import DBF
# from scipy.stats import pearsonr
import scipy.stats as stats
import seaborn as sns
import itertools
import statsmodels.formula.api as smf

ModuleNotFoundError: ignored

# Import Data

In [ ]:
X_his_root = 'Processed Data/IndependentVariableX/HistoricalData'
def readData(path):
     # file root
    table = DBF(path, encoding='GBK')
    df = pd.DataFrame(iter(table))
    return df
POP_his_df={}
LULC_his_df={}
LST_his_df={}
DEM_his_df={}

In [ ]:
for i in range(2005,2021,5):
    path1=X_his_root+'/Pop_HIS_Table/Proj_usa_ppp_'+str(i)+'_UNadj_Table.dbf'
    path2='Processed Data/因变量LST/LST'+str(i)+'.dbf'# LST
    if i==2010:
        path3=X_his_root+'/LULC_HIS_Table/reclass_nlcd_'+str(i)+'.dbf'
    else:
        path3=X_his_root+'/LULC_HIS_Table/reclass_nlcd_'+str(i+1)+'.dbf'
    POP_his_df[str(i)]=readData(path1)
    LST_his_df[str(i)]=readData(path2)
    LULC_his_df[str(i)]=readData(path3)
    DEM_his_df[str(i)]=readData('Processed Data/IndependentVariableX/HistoricalData/DEM_Table/DEM_Table.dbf')

In [ ]:
# DEM_his_df=readData('Processed Data/IndependentVariableX/HistoricalData/DEM_Table/DEM_Table.dbf')

# Clean Data

In [ ]:
len(DEM_his_df['2005'])

34258

In [ ]:
POP_his_df['2005'].head(3)

,FID_,COUNT,AREA,SUM
0,0,100.0,1000000.0,44.800169
1,1,100.0,1000000.0,2.065253
2,2,100.0,1000000.0,88.943340


In [ ]:
POP_his_df['2010'].describe()

,FID_,COUNT,AREA,SUM
count,32527.000000,32527.000000,32527.000000,32527.000000
mean,17929.078058,98.465552,984655.516955,164.845943
std,9498.150608,8.615041,86150.413335,496.928144
min,0.000000,1.000000,10000.000000,0.000000
25%,9861.500000,100.000000,1000000.000000,4.263271
50%,17993.000000,100.000000,1000000.000000,22.164169
75%,26124.500000,100.000000,1000000.000000,92.479616
max,34257.000000,100.000000,1000000.000000,11186.639513


In [ ]:
len(POP_his_df['2005'])

32527

列名SUM表示此格网内人口总数

In [ ]:
LULC_his_df['2005'].head(3)

,FID_,VALUE_1,VALUE_2,VALUE_3,VALUE_4,VALUE_5,VALUE_6,VALUE_7,VALUE_8
0,0,297900.0,3600.0,108000.0,63900.0,0.0,90000.0,0.0,416700.0
1,1,25200.0,0.0,51300.0,0.0,0.0,216900.0,0.0,686700.0
2,2,434700.0,0.0,140400.0,4500.0,0.0,13500.0,0.0,416700.0


In [ ]:
LULC_his_df['2005'].describe()

,FID_,VALUE_1,VALUE_2,VALUE_3,VALUE_4,VALUE_5,VALUE_6,VALUE_7,VALUE_8
count,34258.000000,3.425800e+04,3.425800e+04,34258.000000,3.425800e+04,34258.000000,3.425800e+04,34258.000000,3.425800e+04
mean,17128.500000,1.823214e+05,7.757843e+04,156059.679491,4.644216e+04,3180.343277,4.436418e+05,3944.599802,8.682218e+04
std,9889.577097,2.411756e+05,2.357550e+05,187904.780206,1.243183e+05,22771.232793,3.471851e+05,16229.791803,1.704936e+05
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,8564.250000,0.000000e+00,0.000000e+00,28800.000000,0.000000e+00,0.000000,9.450000e+04,0.000000,0.000000e+00
50%,17128.500000,5.310000e+04,0.000000e+00,84600.000000,1.800000e+03,0.000000,4.149000e+05,0.000000,1.350000e+04
75%,25692.750000,3.078000e+05,9.900000e+03,203400.000000,2.070000e+04,0.000000,7.749000e+05,1800.000000,8.730000e+04
max,34257.000000,1.033200e+06,1.040400e+06,982800.000000,1.035000e+06,543600.000000,1.040400e+06,522000.000000,1.040400e+06


In [ ]:
len(LULC_his_df['2010'])

32565

列名Value 1等表示分类（参考分类对照表），列FID作为唯一标识符。表格内的值表示一个格网内此Landuse类型的占地面积（平方米）。

The column name Value 1, etc. indicates the classification (refer to the Classification Comparison Table) and the column FID is used as a unique identifier. The values in the table represent the area (in square meters) of this Landuse type within a grid.

In [ ]:
DEM_his_df['2005'].head(3)

,FID_,COUNT,AREA,MEAN
0,0,1122.0,1009800.0,14.010071
1,1,1089.0,980100.0,13.740351
2,2,1089.0,980100.0,13.823337


In [ ]:
DEM_his_df['2005'].describe()

,FID_,COUNT,AREA,MEAN
count,34258.000000,34258.000000,3.425800e+04,34258.000000
mean,17128.500000,1111.116323,1.000005e+06,223.690762
std,9889.577097,22.226164,2.000355e+04,209.153268
min,0.000000,1089.000000,9.801000e+05,-1.298297
25%,8564.250000,1089.000000,9.801000e+05,29.589671
50%,17128.500000,1122.000000,1.009800e+06,157.038539
75%,25692.750000,1122.000000,1.009800e+06,394.806578
max,34257.000000,1156.000000,1.040400e+06,1130.898398


In [ ]:
LST_his_df['2005'].head(3)

,FID_,COUNT,AREA,MEAN
0,0,1.0,1000000.0,302.282
1,1,1.0,1000000.0,301.188
2,2,1.0,1000000.0,301.730


In [ ]:
LST_his_df['2020'].describe()

,FID_,COUNT,AREA,MEAN
count,34258.000000,34258.0,34258.0,32265.000000
mean,17128.500000,1.0,1000000.0,301.741838
std,9889.577097,0.0,0.0,3.708273
min,0.000000,1.0,1000000.0,292.692000
25%,8564.250000,1.0,1000000.0,298.662000
50%,17128.500000,1.0,1000000.0,301.678000
75%,25692.750000,1.0,1000000.0,303.968000
max,34257.000000,1.0,1000000.0,314.885000


POP中,LULC中数据不全，LST里有缺失值

## 删除缺失值


In [ ]:
index_l=list(set(POP_his_df['2005']['FID_']) & set(LULC_his_df['2005']['FID_']))

In [ ]:
LULC_his_df[str(i)].iloc[:,1:].sum(axis=1)

0         980100.0
1         980100.0
2        1009800.0
3         980100.0
4         980100.0
           ...    
34253    1009800.0
34254    1009800.0
34255     980100.0
34256     980100.0
34257    1009800.0
Length: 34258, dtype: float64

In [ ]:
for i in range(2005,2021,5):
    LST_his_df[str(i)].dropna(inplace=True)
    LST_his_df[str(i)].reset_index(drop=True,inplace=True)

    LULC_his_df[str(i)]['Row_sum'] = LULC_his_df[str(i)].iloc[:,1:].sum(axis=1)  # 对1~8列按行求和，生成新列
    LULC_his_df[str(i)]=LULC_his_df[str(i)][(LULC_his_df[str(i)]['Row_sum']>=980100)]
    LULC_his_df[str(i)].reset_index(drop=True,inplace=True)

    index_l=list(set(POP_his_df[str(i)]['FID_']) & set(LULC_his_df[str(i)]['FID_'])& set(LST_his_df[str(i)]['FID_'])& set(DEM_his_df[str(i)]['FID_']))
    LST_his_df[str(i)]=LST_his_df[str(i)][LST_his_df[str(i)]['FID_'].isin(index_l)].reset_index(drop=True)
    LULC_his_df[str(i)]=(LULC_his_df[str(i)][LULC_his_df[str(i)]['FID_'].isin(index_l)]).reset_index(drop=True)
    POP_his_df[str(i)]=(POP_his_df[str(i)][POP_his_df[str(i)]['FID_'].isin(index_l)]).reset_index(drop=True)
    DEM_his_df[str(i)] = (DEM_his_df[str(i)][DEM_his_df[str(i)]['FID_'].isin(index_l)]).reset_index(drop=True)

## Describe Data

In [ ]:
for i in ['2005','2010','2015','2020']:
    print(i+"\n")
    print("LST_his_df\n")
    print(LST_his_df[i].describe())
    print("LULC_his_df\n")
    print(LULC_his_df[i].describe())
    print("POP_his_df\n")
    print(POP_his_df[i].describe())
    print("DEM_his_df\n")
    print(DEM_his_df[i].describe())
    print("\n\n")

2005

LST_his_df

             FID_    COUNT       AREA        MEAN
count  32252.0000  32252.0    32252.0  32252.0000
mean   18042.2768      1.0  1000000.0    301.7662
std     9455.9574      0.0        0.0      3.4635
min        0.0000      1.0  1000000.0    293.6067
25%    10067.7500      1.0  1000000.0    298.8700
50%    18130.5000      1.0  1000000.0    301.4720
75%    26193.2500      1.0  1000000.0    304.0320
max    34257.0000      1.0  1000000.0    315.9444
LULC_his_df

             FID_     VALUE_1     VALUE_2      VALUE_3     VALUE_4  \
count  32252.0000  3.2252e+04  3.2252e+04   32252.0000  3.2252e+04   
mean   18042.2768  1.9352e+05  2.3196e+04  165323.0962  4.8811e+04   
std     9455.9574  2.4415e+05  8.1707e+04  189441.8328  1.2705e+05   
min        0.0000  0.0000e+00  0.0000e+00       0.0000  0.0000e+00   
25%    10067.7500  1.8000e+03  0.0000e+00   40500.0000  0.0000e+00   
50%    18130.5000  7.0200e+04  0.0000e+00   91800.0000  1.8000e+03   
75%    26193.2500  3.2940e+05

# Correlation analysis

Correlation analysis: Calculate Pearson correlation of LST vs pop, DEM and LandUse Type Percentage for each pixel. And compare the difference between urban area and rural. And choose the most correlated variables as features in the model. (((如果不是对每个pixel做回归，而是对整体的城市区域和农村区域， if 残差具有空间异质性可以验证一下，LST可能会有空间自相关（Moran指数. 有的话用地理探测器选择因素。

## 2020

In [ ]:
# for i in range(2005,2021,5):
# POP_his_df={}
# LULC_his_df={}
# LST_his_df={}
# data_2020=pd.DataFrame({
#     "LST":LST_his_df['2020']['MEAN'],
#     "LULC_1":LULC_his_df['2020']["VALUE_1"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_2":LULC_his_df['2020']["VALUE_2"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_3":LULC_his_df['2020']["VALUE_3"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_4":LULC_his_df['2020']["VALUE_4"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_5":LULC_his_df['2020']["VALUE_5"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_6":LULC_his_df['2020']["VALUE_6"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_7":LULC_his_df['2020']["VALUE_7"]/LULC_his_df['2020']["Row_sum"],
#     "LULC_8":LULC_his_df['2020']["VALUE_8"]/LULC_his_df['2020']["Row_sum"],
#     "DEM":DEM_his_df['2020']['MEAN'],
#     "POP":POP_his_df['2020']['SUM'],
# })
data_2020=pd.DataFrame({
    "LST":LST_his_df['2020']['MEAN'],
    "LULC_1":LULC_his_df['2020']["VALUE_1"],
    "LULC_2":LULC_his_df['2020']["VALUE_2"],
    "LULC_3":LULC_his_df['2020']["VALUE_3"],
    "LULC_4":LULC_his_df['2020']["VALUE_4"],
    "LULC_5":LULC_his_df['2020']["VALUE_5"],
    "LULC_6":LULC_his_df['2020']["VALUE_6"],
    "LULC_7":LULC_his_df['2020']["VALUE_7"],
    "LULC_8":LULC_his_df['2020']["VALUE_8"],
    "DEM":DEM_his_df['2020']['MEAN'],
    "POP":POP_his_df['2020']['SUM'],
})
List=list(itertools.combinations(data_2020.keys(),2))


In [ ]:
data_2020.describe()

,LST,LULC_1,LULC_2,LULC_3,LULC_4,LULC_5,LULC_6,LULC_7,LULC_8,DEM,POP
count,32252.0000,3.2252e+04,3.2252e+04,32252.0000,3.2252e+04,32252.0000,3.2252e+04,32252.0000,3.2252e+04,32252.0000,32252.0000
mean,301.7413,1.8677e+05,2.2280e+04,167607.1406,5.5613e+04,3074.7116,4.6834e+05,4751.6495,9.1565e+04,237.5859,170.0050
std,3.7087,2.3945e+05,8.1288e+04,189997.4336,1.3410e+05,20846.1141,3.3972e+05,18240.9669,1.7288e+05,207.7608,495.5332
min,292.6920,0.0000e+00,0.0000e+00,0.0000,0.0000e+00,0.0000,0.0000e+00,0.0000,0.0000e+00,-1.2567,0.0000
25%,298.6620,1.8000e+03,0.0000e+00,40500.0000,0.0000e+00,0.0000,1.3320e+05,0.0000,0.0000e+00,45.0098,4.2118
50%,301.6780,6.4800e+04,0.0000e+00,92700.0000,3.6000e+03,0.0000,4.5990e+05,0.0000,1.8900e+04,172.8586,23.7331
75%,303.9680,3.1252e+05,4.5000e+03,227700.0000,3.1500e+04,0.0000,7.8930e+05,3600.0000,9.5400e+04,405.0378,100.4023
max,314.8850,1.0053e+06,1.0404e+06,981000.0000,1.0350e+06,611100.0000,1.0404e+06,786600.0000,1.0404e+06,1130.8984,10825.6673


In [ ]:
Corr_2020=data_2020.corr(method='pearson')
Corr_2020

,LST,LULC_1,LULC_2,LULC_3,LULC_4,LULC_5,LULC_6,LULC_7,LULC_8,DEM,POP
LST,1.0000,0.1939,-0.0134,0.6142,0.6114,0.0368,-0.7453,-0.0016,0.0489,-0.7988,0.5086
LULC_1,0.1939,1.0000,-0.1398,-0.1201,-0.1766,-0.0423,-0.4813,-0.0827,-0.0883,-0.3216,-0.1616
LULC_2,-0.0134,-0.1398,1.0000,-0.0986,-0.0182,0.0603,-0.1541,0.0378,0.1386,-0.0894,-0.0499
LULC_3,0.6142,-0.1201,-0.0986,1.0000,0.3861,-0.0413,-0.5041,-0.0642,-0.1821,-0.3932,0.4142
LULC_4,0.6114,-0.1766,-0.0182,0.3861,1.0000,0.0148,-0.4283,-0.0078,-0.1051,-0.3190,0.7148
LULC_5,0.0368,-0.0423,0.0603,-0.0413,0.0148,1.0000,-0.0351,0.2675,-0.0153,-0.0252,-0.0273
LULC_6,-0.7453,-0.4813,-0.1541,-0.5041,-0.4283,-0.0351,1.0000,0.0196,-0.3348,0.7526,-0.3303
LULC_7,-0.0016,-0.0827,0.0378,-0.0642,-0.0078,0.2675,0.0196,1.0000,-0.0028,0.0173,-0.0457
LULC_8,0.0489,-0.0883,0.1386,-0.1821,-0.1051,-0.0153,-0.3348,-0.0028,1.0000,-0.3106,-0.1057
DEM,-0.7988,-0.3216,-0.0894,-0.3932,-0.3190,-0.0252,0.7526,0.0173,-0.3106,1.0000,-0.2540


In [ ]:
# sns.heatmap(Corr_2020,linewidths=0.8,vmax=1.0, square=True,linecolor='white', annot=True)

In [ ]:
p_2020= pd.DataFrame(columns=Corr_2020.columns,index=Corr_2020.index)
p_2020

,LST,LULC_1,LULC_2,LULC_3,LULC_4,LULC_5,LULC_6,LULC_7,LULC_8,DEM,POP
LST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in List:
    i=i+stats.pearsonr(data_2020[i[0]],data_2020[i[1]])
    p_2020[i[0]][i[1]]=i[3]
    # print(i)

In [ ]:
pd.options.display.precision = 4
p_2020

,LST,LULC_1,LULC_2,LULC_3,LULC_4,LULC_5,LULC_6,LULC_7,LULC_8,DEM,POP
LST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_2,0.0162,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_3,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_4,0.0,0.0,0.0011,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LULC_5,0.0,0.0,0.0,0.0,0.0081,NaN,NaN,NaN,NaN,NaN,NaN
LULC_6,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
LULC_7,0.769,0.0,0.0,0.0,0.1595,0.0,0.0004,NaN,NaN,NaN,NaN
LULC_8,0.0,0.0,0.0,0.0,0.0,0.006,0.0,0.6105,NaN,NaN,NaN
DEM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0019,0.0,NaN,NaN


In [ ]:
f1='LST ~ '+'+'.join(data_2020.columns[1:])
f1

'LST ~ LULC_1+LULC_2+LULC_3+LULC_4+LULC_5+LULC_6+LULC_7+LULC_8+DEM+POP'

VIF值越大则共线性问题越明显，通常以10作为判断边界。当VIF<10不存在多重共线性；当10<=VIF<100,存在较强的多重共线性；当VIF>=100, 存在严重多重共线性。

In [ ]:
reg = smf.ols(formula=f1, data = data_2020)
results = reg.fit()
from statsmodels.stats.outliers_influence import variance_inflation_factor

exog = reg.exog #获取自变量矩阵（第一列为截距项）
exog_names = reg.exog_names #获取自变量名称（第一个为截距项）

vif = []
for i in range(exog.shape[1]-1):
    vif.append(variance_inflation_factor(exog,i+1))
    print(f'{exog_names[i+1]} 的方差膨胀因子VIF = {vif[i]}')

LULC_1 的方差膨胀因子VIF = 144.14227877835953
LULC_2 的方差膨胀因子VIF = 17.502931109842063
LULC_3 的方差膨胀因子VIF = 91.44454336500473
LULC_4 的方差膨胀因子VIF = 46.84565628833994
LULC_5 的方差膨胀因子VIF = 2.1664312645460804
LULC_6 的方差膨胀因子VIF = 289.5353016532413
LULC_7 的方差膨胀因子VIF = 1.9190702608056303
LULC_8 的方差膨胀因子VIF = 75.82014814283376
DEM 的方差膨胀因子VIF = 2.3445221321747725
POP 的方差膨胀因子VIF = 2.158312413794147


LULC_1, LULC_6 >100. so delete L6

In [ ]:
f2='LST ~ '+'+'.join(data_2020.columns[1:6].append(data_2020.columns[7:]))
f2

'LST ~ LULC_1+LULC_2+LULC_3+LULC_4+LULC_5+LULC_7+LULC_8+DEM+POP'

In [ ]:
reg2 = smf.ols(formula=f2, data = data_2020)
results = reg2.fit()
from statsmodels.stats.outliers_influence import variance_inflation_factor

exog = reg2.exog #获取自变量矩阵（第一列为截距项）
exog_names = reg2.exog_names #获取自变量名称（第一个为截距项）

vif = []
for i in range(exog.shape[1]-1):
    vif.append(variance_inflation_factor(exog,i+1))
    print(f'{exog_names[i+1]} 的方差膨胀因子VIF = {vif[i]}')


LULC_1 的方差膨胀因子VIF = 1.6430499558570708
LULC_2 的方差膨胀因子VIF = 1.0970213538158042
LULC_3 的方差膨胀因子VIF = 1.7327492560382998
LULC_4 的方差膨胀因子VIF = 2.290769233055511
LULC_5 的方差膨胀因子VIF = 1.0890244446998605
LULC_7 的方差膨胀因子VIF = 1.092735607828395
LULC_8 的方差膨胀因子VIF = 1.5296752653856176
DEM 的方差膨胀因子VIF = 2.341242640390892
POP 的方差膨胀因子VIF = 2.157778959059477


## Reprojection

## Resample